In [4]:
import pandas as pd

df = pd.read_json("../_datasets/death-reports.json")

In [5]:
duplicated_df = df[df["id"].duplicated()].iloc[:, [0, 2, 3]]
if len(duplicated_df) > 0:
	print("[Error] idが重複したデータがあります。修正してください。")
	print(duplicated_df)